In [1]:
import pandas as pd
import json
import numpy as np
import datetime
from clickhouse_driver import Client
client = Client(host='localhost', settings={'use_numpy': True})

In [3]:
micro=['648.exchange2_s', '607.cactuBSSN_s', '505.mcf_r', '508.namd_r',
   '627.cam4_s', '619.lbm_s', '631.deepsjeng_s',
   '526.blender_r', '520.omnetpp_r', '511.povray_r', '500.perlbench_r',
   '507.cactuBSSN_r', '523.xalancbmk_r',
   '510.parest_r', '603.bwaves_s', '649.fotonik3d_s', '557.xz_r',
   '527.cam4_r', '531.deepsjeng_r', '623.xalancbmk_s', '525.x264_r',
   '605.mcf_s', '644.nab_s', '519.lbm_r', '538.imagick_r', '641.leela_s',
   '628.pop2_s', '638.imagick_s', '541.leela_r', '625.x264_s', '602.gcc_s',
   '600.perlbench_s', '620.omnetpp_s', '621.wrf_s', '657.xz_s',
   '521.wrf_r', '502.gcc_r', '503.bwaves_r', '654.roms_s']
app2score={'Perl interpreter': ['500.perlbench_r', '600.perlbench_s'],
    'GNU C compiler': ['502.gcc_r', '602.gcc_s'],
    'Route planning': ['505.mcf_r', '605.mcf_s'],
    'Discrete Event simulation - computer network': ['520.omnetpp_r',
    '620.omnetpp_s'],
    'XML to HTML conversion via XSLT': ['523.xalancbmk_r', '623.xalancbmk_s'],
    'Video compression': ['525.x264_r', '625.x264_s'],
    'Artificial Intelligence: alpha-beta tree search (Chess)': ['531.deepsjeng_r',
    '631.deepsjeng_s'],
    'Artificial Intelligence: Monte Carlo tree search (Go)': ['541.leela_r',
    '641.leela_s'],
    'Artificial Intelligence: recursive solution generator (Sudoku)': ['648.exchange2_s'],
    'General data compression': ['557.xz_r', '657.xz_s'],
    'Explosion modeling': ['503.bwaves_r', '603.bwaves_s'],
    'Physics: relativity': ['507.cactuBSSN_r', '607.cactuBSSN_s'],
    'Molecular dynamics': ['508.namd_r', '644.nab_s'],
    'Biomedical imaging: optical tomography with finite elements': ['510.parest_r'],
    'Ray tracing': ['511.povray_r'],
    'Fluid dynamics': ['519.lbm_r', '619.lbm_s'],
    'Weather forecasting': ['521.wrf_r', '621.wrf_s'],
    '3D rendering and animation': ['526.blender_r'],
    'Atmosphere modeling': ['527.cam4_r', '627.cam4_s'],
    'Wide-scale ocean modeling (climate level)': ['628.pop2_s'],
    'Image manipulation': ['538.imagick_r', '638.imagick_s'],
    'Computational Electromagnetics': ['649.fotonik3d_s'],
    'Regional ocean modeling': [ '654.roms_s']}
app2tag = {
    "Perl interpreter": 'perl_tag',
    "GNU C compiler": 'gnu_tag',
    "Route planning": 'route_tag',
    "Discrete Event simulation - computer network": 'discrete_tag',
    "XML to HTML conversion via XSLT": 'xml_tag',
    "Video compression": 'video_tag',
    "Artificial Intelligence: alpha-beta tree search (Chess)": 'aichess_tag',
    "Artificial Intelligence: Monte Carlo tree search (Go)": 'aimc_tag',
    "Artificial Intelligence: recursive solution generator (Sudoku)": 'aisudoku_tag',
    "General data compression": 'compression_tag',
    "Explosion modeling": 'explosion_tag',
    "Physics: relativity": 'physics_tag',
    "Molecular dynamics": 'molecular_tag',
    "Biomedical imaging: optical tomography with finite elements": 'biomedical_tag',
    "Ray tracing": 'ray_tag',
    "Fluid dynamics": 'fluid_tag',
    "Weather forecasting": 'weather_tag', 
    "3D rendering and animation": 'render_3D_tag',
    "Atmosphere modeling": 'atmosphere_tag',
    "Wide-scale ocean modeling (climate level)": 'climateocean_tag',
    "Image manipulation": 'image_tag',
    "Computational Electromagnetics": 'electro_tag',
    "Regional ocean modeling": 'regionalocean_tag'
}

In [12]:
client.execute("""drop  table if exists tmp""")
client.execute("""drop  table if exists tmp2""")
client.execute("""drop  table if exists tmp3""")
client.execute("""drop  table if exists tags""")

[]

In [13]:
print([i[0] for i in client.execute("""show tables""")])

['cpu', 'mach', 'test']


In [14]:
client.execute('''select pow(arrayProduct([1, 2, 3, 4]), 1/length([1, 2 , 3 , 4]))''')

[array([2.21336384])]

In [5]:
# client.execute("""CREATE TABLE default.tmp   ENGINE = Memory AS SELECT * FROM default.test T left join default.mach M
                    # on T.machid=M.machid left join default.cpu C on M.cpuid=C.cpuid""")
print("""CREATE TABLE default.tmp ENGINE = Memory AS SELECT * FROM default.test T left join default.mach M on T.machid=M.machid left join default.cpu C on M.cpuid=C.cpuid""")

# tmp2:cpu分组后micro均值 - yoyo: 使用了几何平均
tmp=','.join(['''pow(arrayProduct(arrayFilter(x -> x != 0, groupArray(divide(`T.{}`, `M.Chips`)))), 1 / length(arrayFilter(x -> x != 0, groupArray(`T.{}`)))) `{}_geomean` '''.format(col,col,col) for col in micro])
# client.execute("""CREATE TABLE tmp2 ENGINE = Memory AS SELECT `C.Processor`,{} FROM tmp GROUP BY `C.Processor`""".format(tmp))
print("""CREATE TABLE tmp2 ENGINE = Memory AS SELECT `C.Processor`,{} FROM tmp GROUP BY `C.Processor`""".format(tmp))

# tmp3:cpu分组后app score均值
tmp=','.join([' + '.join(['`{}_geomean`'.format(x) for x in app2score[k]])+' `{}`'.format(k) for k in app2score.keys()])
# client.execute("""CREATE TABLE tmp3 ENGINE = Memory AS select `C.Processor`,{} from tmp2;""".format(tmp))
print("""CREATE TABLE tmp3 ENGINE = Memory AS select `C.Processor`,{} from tmp2;""".format(tmp))

CREATE TABLE default.tmp ENGINE = Memory AS SELECT * FROM default.test T left join default.mach M on T.machid=M.machid left join default.cpu C on M.cpuid=C.cpuid
CREATE TABLE tmp2 ENGINE = Memory AS SELECT `C.Processor`,pow(arrayProduct(arrayFilter(x -> x != 0, groupArray(divide(`T.648.exchange2_s`, `M.Chips`)))), 1 / length(arrayFilter(x -> x != 0, groupArray(`T.648.exchange2_s`)))) `648.exchange2_s_geomean` ,pow(arrayProduct(arrayFilter(x -> x != 0, groupArray(divide(`T.607.cactuBSSN_s`, `M.Chips`)))), 1 / length(arrayFilter(x -> x != 0, groupArray(`T.607.cactuBSSN_s`)))) `607.cactuBSSN_s_geomean` ,pow(arrayProduct(arrayFilter(x -> x != 0, groupArray(divide(`T.505.mcf_r`, `M.Chips`)))), 1 / length(arrayFilter(x -> x != 0, groupArray(`T.505.mcf_r`)))) `505.mcf_r_geomean` ,pow(arrayProduct(arrayFilter(x -> x != 0, groupArray(divide(`T.508.namd_r`, `M.Chips`)))), 1 / length(arrayFilter(x -> x != 0, groupArray(`T.508.namd_r`)))) `508.namd_r_geomean` ,pow(arrayProduct(arrayFilter(x -> x !

In [23]:
print('Show the infos of 3 tmp tables')
print('tmp:\t', ', '.join([i[0] for i in client.execute("SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name='tmp'")]),
      '\ntmp2:\t', ', '.join([i[0] for i in client.execute("SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name='tmp2'")]),
       '\ntmp3:\t', ', '.join([i[0] for i in client.execute("SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name='tmp3'")]))

Show the infos of 3 tmp tables
tmp:	 T.Benchmark, T.machid, T.testid, T.HW Avail, T.SW Avail, T.License, T.TestedBy, T.TestSponsor, T.TestDate, T.Published, T.Updated, T.PeakResult, T.BaseResult, T.EnergyPeakResult, T.EnergyBaseResult, T.531.deepsjeng_r, T.600.perlbench_s, T.625.x264_s, T.510.parest_r, T.500.perlbench_r, T.502.gcc_r, T.607.cactuBSSN_s, T.623.xalancbmk_s, T.619.lbm_s, T.644.nab_s, T.602.gcc_s, T.505.mcf_r, T.649.fotonik3d_s, T.507.cactuBSSN_r, T.654.roms_s, T.557.xz_r, T.605.mcf_s, T.525.x264_r, T.511.povray_r, T.638.imagick_s, T.621.wrf_s, T.541.leela_r, T.520.omnetpp_r, T.508.namd_r, T.631.deepsjeng_s, T.641.leela_s, T.503.bwaves_r, T.627.cam4_s, T.657.xz_s, T.648.exchange2_s, T.538.imagick_r, T.523.xalancbmk_r, T.548.exchange2_r, T.521.wrf_r, T.519.lbm_r, T.628.pop2_s, T.526.blender_r, T.603.bwaves_s, T.527.cam4_r, T.620.omnetpp_s, M.machid, M.Vendor, M.System, M.Cores, M.Chips, M.Memory, M.MemoSize(GB), M.MemoNum, M.Storage, M.DiskSize(TB), M.SSD, M.OperatingSystem,

In [17]:
print([i[0] for i in client.execute("""show tables""")])

['cpu', 'mach', 'test', 'tmp', 'tmp2', 'tmp3']


In [20]:
q1 = ','.join([''' if(`{}` =='{}', 1, 0) as {} '''.format(app2tag[app_name], app2tag[app_name], app2tag[app_name]) for app_name in app2tag])
q2 = ' left outer join '.join(['''
    (SELECT `C.Processor`, `{}`, '{}' as {}
        FROM tmp3 where `{}` > 
        (select quantile(0.9)(`{}`) from tmp3)
    ) as {}_R on L.`C.Processor` == {}_R.`C.Processor`
'''.format(app_name, app2tag[app_name], app2tag[app_name], app_name, app_name, app2tag[app_name], app2tag[app_name]) for app_name in app2tag])
q = '''create table tags ENGINE = Memory as select L.`C.Processor`,'''
q += q1
q += '''from tmp3 as L left outer join'''
q += q2
q += ''' settings join_use_nulls = 1;'''
print(q)

create table tags ENGINE = Memory as select L.`C.Processor`, if(`perl_tag` =='perl_tag', 1, 0) as perl_tag , if(`gnu_tag` =='gnu_tag', 1, 0) as gnu_tag , if(`route_tag` =='route_tag', 1, 0) as route_tag , if(`discrete_tag` =='discrete_tag', 1, 0) as discrete_tag , if(`xml_tag` =='xml_tag', 1, 0) as xml_tag , if(`video_tag` =='video_tag', 1, 0) as video_tag , if(`aichess_tag` =='aichess_tag', 1, 0) as aichess_tag , if(`aimc_tag` =='aimc_tag', 1, 0) as aimc_tag , if(`aisudoku_tag` =='aisudoku_tag', 1, 0) as aisudoku_tag , if(`compression_tag` =='compression_tag', 1, 0) as compression_tag , if(`explosion_tag` =='explosion_tag', 1, 0) as explosion_tag , if(`physics_tag` =='physics_tag', 1, 0) as physics_tag , if(`molecular_tag` =='molecular_tag', 1, 0) as molecular_tag , if(`biomedical_tag` =='biomedical_tag', 1, 0) as biomedical_tag , if(`ray_tag` =='ray_tag', 1, 0) as ray_tag , if(`fluid_tag` =='fluid_tag', 1, 0) as fluid_tag , if(`weather_tag` =='weather_tag', 1, 0) as weather_tag , if(

In [21]:
client.execute(q)

[]